In [142]:
!wget -q -O rps.zip https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/rps_data_sample.zip
!unzip -qq rps.zip

replace rps_data_sample/scissors/604.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


Download of official tutorial dataset above

extract light weight Hagrid dataset from google drive

In [2]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

!unzip -u "/content/drive/My Drive/cs766_Hagrid.zip" -d "./new_data"

Streaming output truncated to the last 5000 lines.
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/34266f02-173a-4b57-92a6-ec6b31133758.jpg  
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/344022b9-8475-4649-bb2d-0b94701d8a49.jpg  
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/3466c3f4-c9c9-4bfc-8dce-3631935ccf71.jpg  
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/34878ee4-4c76-491b-96d4-554e01247a42.jpg  
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/34bb9230-a0b5-4a6d-a3dd-1409043a7509.jpg  
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/34d37ccb-7110-4ec5-9253-58e2d6b0a530.jpg  
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/34fb8160-3a0d-47e9-82fb-342ce1367a6c.jpg  
  inflating: ./new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_three2/35177a5d-2e74-413a-9670-cfd1a570f9d7.jpg  
  inf

Intall suitable version of media pipe maker

In [5]:
!pip install 'keras<3.0.0' mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 80.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible with other requirements. This could take 

crop Hagrid dataset using its own label

In [135]:
import json
import os

k = 0
class_need = ['palm','fist','one','like']
os.mkdir("./crop_data/")
os.mkdir("./crop_data/none")
none_count = 0
for classes in class_need:
  f = open('new_data/hagrid-sample-30k-384p/ann_train_val/'+classes+'.json')
  d = json.load(f)
  dir_name = './new_data/hagrid-sample-30k-384p/hagrid_30k/train_val_'+classes+'/'
  imgList = os.listdir(dir_name);
  os.mkdir("./crop_data/"+classes+"/")
  class_count = 0;

  for imgName in imgList:
    img = Image.open(dir_name+imgName)
    imgID=imgName[0:(len(imgName)-4)]
    # print(imgID)
    bboxes = d[imgID]['bboxes']
    labels = d[imgID]['labels']
    i = -1
    class_count += 1;
    if(class_count==200): continue
    for bbox in bboxes:
      i+=1;

      min_x = bbox[0]
      min_y = bbox[1]
      max_x = min_x+bbox[2]
      max_y = min_y+bbox[3]
      img = np.array(img)
      min_x*=img.shape[1]
      min_y*=img.shape[0]
      max_x*=img.shape[1]
      max_y*=img.shape[0]
      ans = Image.fromarray(img)

      box = (min_x, min_y, max_x, max_y)
      #print(box)
      ans = ans.crop(box)
      #cv2_imshow(cv2.cvtColor(np.array(ans), cv2.COLOR_BGR2RGB))
      if(labels[i]=="no_gesture"):
        if(none_count==200):continue
        cv2.imwrite("./crop_data/none/"+imgID+str(i)+".jpg", cv2.cvtColor(np.array(ans), cv2.COLOR_BGR2RGB))
        none_count+=1;
        continue;
      else:
        cv2.imwrite("./crop_data/"+classes+"/"+imgID+str(i)+".jpg", cv2.cvtColor(np.array(ans), cv2.COLOR_BGR2RGB))


  f.close()

remove the dataset directory for testing purpose

In [134]:
import shutil
shutil.rmtree("./crop_data/")

combining two datasets

In [136]:
src="./rps_data_sample/paper/"
dst="./crop_data/palm/"
file_list = os.listdir(src);
for file_name in file_list:
  shutil.move(src+file_name,dst+file_name)

In [137]:
src="./rps_data_sample/rock/"
dst="./crop_data/fist/"
file_list = os.listdir(src);
for file_name in file_list:
  shutil.move(src+file_name,dst+file_name)

get official hand detection model for croping of frame

In [154]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [155]:
!wget -q -O image.jpg https://storage.googleapis.com/mediapipe-tasks/hand_landmarker/woman_hands.jpg

import cv2
from google.colab.patches import cv2_imshow


Turning result into human understandable version. Source: official tutorial:https://github.com/googlesamples/mediapipe/tree/main

In [47]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]

    # print(x_coordinates)
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

Cropping the bbox references to code above

In [102]:
from PIL import Image

def crop_detection(rgb_image, detection_result,fname):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness

  # Loop through the detected hands to visualize.
  annotated_image = np.copy(rgb_image.numpy_view())
  ans = [];
  margin_err = 0.13
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    height, width, _ = annotated_image.shape
    x_coordinates=[]
    y_coordinates=[]
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    # print(min(x_coordinates))
    # print(min(y_coordinates))
    # print(max(x_coordinates))
    # print(max(y_coordinates))
    min_x = (min(x_coordinates) * width)
    min_y = (min(y_coordinates) * height)
    max_x = (max(x_coordinates) * width)
    max_y = (max(y_coordinates) * height)
    #cv2.rectangle(annotated_image, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
    #cv2_imshow(annotated_image)


    # print(hand_landmarks_list[idx])
    # print(x_coordinates)
    # print(min_x)
    # print(min_y)
    # print(max_x)
    # print(max_y)

    w = max_x-min_x
    h = max_y-min_y
    min_x -= w*margin_err
    min_x = max(0,min_x)
    min_y -= h*margin_err
    min_y = max(0,min_y)
    max_x += w*margin_err
    max_x = min(width,max_x)
    max_y += h*margin_err
    max_y = min(height,max_y)

    # print(annotated_image.shape)
    box = (min_x, min_y, max_x, max_y)
    ans = Image.fromarray(annotated_image)
    ans = ans.crop(box)
    # cv2_imshow(cv2.cvtColor(np.array(ans), cv2.COLOR_BGR2RGB))
    cv2.imwrite("./crop_frame/"+fname+str(idx)+".png", cv2.cvtColor(np.array(ans), cv2.COLOR_BGR2RGB))

Cropping out frame

In [101]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=10)
detector = vision.HandLandmarker.create_from_options(options)
new_dir = "./output_frame/"
# Load the input image.
imgList = os.listdir(new_dir);
total = 0
correct = 0
imgList.sort()
print(imgList)
shutil.rmtree("./crop_frame/")
os.mkdir("./crop_frame")
for imgName in imgList:
# STEP 3: Load the input image.
  image = mp.Image.create_from_file("./output_frame/"+imgName)#"./new_data/leapGestRecog/00/01_palm/frame_00_01_0003.png")
  imgID=imgName[0:(len(imgName)-4)]
# STEP 4: Detect hand landmarks from the input image.
  detection_result = detector.detect(image)

# STEP 5: Process the classification result. In this case, visualize it.
  crop_detection(image, detection_result,fname=imgID)


['frame100_0min21s.jpg', 'frame101_0min22s.jpg', 'frame102_0min22s.jpg', 'frame103_0min22s.jpg', 'frame104_0min22s.jpg', 'frame105_0min22s.jpg', 'frame106_0min23s.jpg', 'frame107_0min23s.jpg', 'frame108_0min23s.jpg', 'frame109_0min23s.jpg', 'frame10_0min2s.jpg', 'frame110_0min24s.jpg', 'frame111_0min24s.jpg', 'frame112_0min24s.jpg', 'frame113_0min24s.jpg', 'frame114_0min24s.jpg', 'frame115_0min25s.jpg', 'frame116_0min25s.jpg', 'frame117_0min25s.jpg', 'frame118_0min25s.jpg', 'frame119_0min26s.jpg', 'frame11_0min2s.jpg', 'frame120_0min26s.jpg', 'frame121_0min26s.jpg', 'frame122_0min26s.jpg', 'frame123_0min26s.jpg', 'frame124_0min27s.jpg', 'frame125_0min27s.jpg', 'frame126_0min27s.jpg', 'frame127_0min27s.jpg', 'frame128_0min28s.jpg', 'frame129_0min28s.jpg', 'frame12_0min2s.jpg', 'frame130_0min28s.jpg', 'frame131_0min28s.jpg', 'frame132_0min29s.jpg', 'frame133_0min29s.jpg', 'frame134_0min30s.jpg', 'frame135_0min30s.jpg', 'frame136_0min30s.jpg', 'frame137_0min31s.jpg', 'frame138_0min31s.jpg

Import gesture_recognizer

In [10]:
from mediapipe_model_maker import gesture_recognizer

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


import dataset and split into three pieces for cross validation

In [138]:
# Load the rock-paper-scissor image archive.
data = gesture_recognizer.Dataset.from_folder(
    dirname="./crop_data/",#IMAGES_PATH,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

# Split the archive into training, validation and test dataset.
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder


Train model

In [139]:
# Train the model
hparams = gesture_recognizer.HParams(export_dir="rock_paper_scissors_model", batch_size =8,
    epochs=10,)
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_7 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_7 (ReLU)              (None, 128)               0         
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 5)                 645       
 out (Dense)                                                     
                                                           

Test model

In [140]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

302/302 [==============================] - 3s 2ms/step - loss: 0.0641 - categorical_accuracy: 0.9636
Test loss:0.06411716341972351, Test accuracy:0.9635761380195618


export model

In [146]:
# Export the model bundle.
model.export_model()

# Rename the file to be more descriptive.
!mv rock_paper_scissors_model/gesture_recognizer.task rock_paper_scissors.task

Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite


download model

In [153]:
from google.colab import files
files.download("rock_paper_scissors.task")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [156]:
from google.colab.patches import cv2_imshow
import cv2

just another detection test, user can ignore this part

In [148]:
# detection test
# ==========================================================================================
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os

# Create a GestureRecognizer object.
model_path = os.path.abspath("rock_paper_scissors.task")
recognizer = vision.GestureRecognizer.create_from_model_path(model_path)
new_dir = "./rps_data_sample/rock/"
# Load the input image.
imgList = os.listdir(new_dir);
total = 0
correct = 0
for imgName in imgList:
  total+=1
  image = mp.Image.create_from_file(new_dir+imgName)#'rps_data_sample/scissors/117.jpg')#7.jpg')#'rps_data_sample/scissors/100.jpg')

# Run gesture recognition.
  recognition_result = recognizer.recognize(image)
  # print(recognition_result)

# Display the most likely gesture.
  if(recognition_result.gestures == []):continue
  top_gesture = recognition_result.gestures[0][0]
  if(top_gesture.category_name=="fist"):#"fist" or top_gesture.category_name=="like"):
    correct+=1;
print(correct/total)
#print(f"Gesture recognized: {top_gesture.category_name} ({top_gesture.score})")

0.84


Frame extraction

In [152]:
#frame extraction
import time
cap = cv2.VideoCapture('1.mov')
size = (
	int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
	int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
# codec = cv2.VideoWriter_fourcc(*'DIVX')
# output = cv2.VideoWriter('human2.avi',codec,25.0,size)
shutil.rmtree("./output_frame/")
os.mkdir("./output_frame")

i = 0
frame_rate_divider = 1
while(cap.isOpened()):
    stime = time.time()
    ret, frame = cap.read()
    i += 1
    #if(i<=150):continue
    if ret:
        if i % frame_rate_divider == 0:
            ms = cap.get(cv2.CAP_PROP_POS_MSEC);
            s = ms/1000;
            s = int(s)
            mins = int(s/60)
            s = s-mins*60
            cv2.imwrite("output_frame/frame"+str(i)+"_"+str(mins)+"min"+str(s)+"s"+".jpg",frame)
            # output.write(frame)
        # print('FPS {:.1f}'.format(1 / (time.time() - stime)))
        if(i==808):break;
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
# output.release()
cv2.destroyAllWindows()


In [93]:
import shutil
shutil.rmtree("./output_frame/")

Detection on frame

In [151]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os

# Create a GestureRecognizer object.
model_path = os.path.abspath("rock_paper_scissors.task")
recognizer = vision.GestureRecognizer.create_from_model_path(model_path)
new_dir = "./output_frame/"
# Load the input image.
imgList = os.listdir(new_dir);
imgList.sort()
total = 0
correct = 0
for imgName in imgList:
  total+=1
  image = mp.Image.create_from_file(new_dir+imgName)#'rps_data_sample/scissors/117.jpg')#7.jpg')#'rps_data_sample/scissors/100.jpg')

# Run gesture recognition.
  recognition_result = recognizer.recognize(image)
  # print(recognition_result)

# Display the most likely gesture.
  if(recognition_result.gestures == []):continue
  top_gesture = recognition_result.gestures[0][0]
  if(not (top_gesture.category_name=="none" or top_gesture.category_name=="")):
    print(imgName)
    print(top_gesture.category_name)
# print(correct/total)
#print(f"Gesture recognized: {top_gesture.category_name} ({top_gesture.score})")

frame101_0min22s.jpg
fist
frame127_0min27s.jpg
like
frame128_0min28s.jpg
like
frame149_0min35s.jpg
like
frame241_1min4s.jpg
fist
frame242_1min4s.jpg
fist
frame245_1min5s.jpg
fist
frame284_1min13s.jpg
fist
frame285_1min14s.jpg
fist
frame286_1min14s.jpg
like
frame327_1min23s.jpg
like
frame328_1min23s.jpg
like
frame329_1min23s.jpg
like
frame330_1min23s.jpg
like
frame331_1min23s.jpg
like
frame332_1min24s.jpg
like
frame333_1min24s.jpg
like
frame335_1min24s.jpg
fist
frame337_1min25s.jpg
like
frame412_1min42s.jpg
like
frame421_1min44s.jpg
like
frame433_1min47s.jpg
like
frame442_1min49s.jpg
fist
frame443_1min49s.jpg
fist
frame444_1min49s.jpg
like
frame501_2min5s.jpg
like
frame684_2min55s.jpg
like
frame688_2min56s.jpg
fist
frame689_2min56s.jpg
like
frame690_2min56s.jpg
fist
frame691_2min57s.jpg
like
frame692_2min57s.jpg
like
frame693_2min57s.jpg
like
frame694_2min57s.jpg
like
frame695_2min58s.jpg
like
frame699_2min58s.jpg
like
frame701_2min59s.jpg
like
frame702_2min59s.jpg
like
frame703_2min59s